# Introduction

While in an ideal world, we could jump right into analyze the data, the data that we receive is not always in a condition to be actually usable. So now that we dealt with the missing data, let's look to see if there is any data that needs to be cleaned up. This can include data that is inconsistent, such as inconsistent spelling or capitalization of strings, or invalid, such as a sensor measurement that falls outside of feasible real-world values.

In most situations and with most datasets, this part of the process will take a significant chunk of time and energy - often more than the analysis itself - as real world data tends to be messy. But confidence in the results and conclusions of your analysis requires confidence in your data. In this case, we have done a lot of the work to clean up this data set already. We've left a few things in there for you to address, but expect that it will be this quick and easy when working with a real-world dataset. 

The goal at the end of this notebook is to end up with dataset that is clean and ready for analysis.

# Ingesting Data + Prep

You'll notice that at the beginning of each notebook, there is a code section for importing libraries. Libraries are reusable functions and methods also known as packages. When installing Python, there is a standard library which comes with it. For more information about the Standard Library, see https://docs.python.org/3/library/.

However, there are some very well known libraries that are extremely useful for data wrangling and analysis that we have used and continue to use. One of the most used libraries for analysts is pandas (for more information about pandas, see https://pandas.pydata.org/). Typically, for these non-prepackaged libraries, we need to install them and import them. Installation only needs to be done once in your environment, but in each notebook we need to reimport each library each time we start up the notebook.

In [ ]:
# Import Libraries

import pandas as pd
import numpy as np
import datetime

Now that we have imported the data a few times. We will leave it up to you. The dataset is stored in a csv called complete_data.csv. Follow the folder structure from previous notebooks.

In [ ]:
# Answer
filepath = r'/home/jovyan/complete_data.csv'
df = pd.read_csv(filepath)

# Cleaning the Data

One of the most common issues with new data sets is duplicate data. Thus, this is generally one of the first things to check for. Let's see if this dataset has duplicates and if there are then let'ss find out how many duplicates there are. 

We can do this using the duplicated() method on our dataframe. Why don't you give it a try?

In [ ]:
# Answer

index = df.duplicated()

df[index].head()

Looks like we found some duplicates. How many are there?

In [ ]:
# Answer

count_of_dups = df.duplicated().sum()

print('The number of duplicates is', count_of_dups)

Alright, so there are a lot of duplicates in our data. Let's remove them. We can do this with the drop_duplicates() method. Give it a try below. Then double check that all duplicates have been removed. Hint: the syntax is similar to the duplicated() and isnull() method.

In [ ]:
# Answer

dedup_df = df.drop_duplicates()

dedup_df.duplicated().sum()

Now that we have removed the duplicates, lets move on to check if the data covers the desired timeframe. We know that the data is supposed to be from 1-1-2020 to 9-1-2020. Let's check if this is actually the case.

Let's subset the data and see if there there are any observations that are before 1-1-2020 or after 9-1-2020. 

In [ ]:
# Answer
index = dedup_df['time'] < '2020-01-01'
dedup_df[index]

So we see that there are some events that occur before 1/1/20. That might throw off our things like total event counts, so we will remove this data so that our dataset aligns with the scope of our analysis. 

In [ ]:
# Answer
index = dedup_df['time'] >= '2020-01-01'
timed_df = dedup_df[index].copy()

Another helpful check is to plot the event counts with a line graph to make sure that there are no unexpected gaps in the middle of the data. We'll save that for when we introduce plotting in the next notebook.

Categorical data is data that only involves a limited or fixed number of possible values. The event column is an example of categorical data that we have already cleaned - the only possible values are "on" or "off." Another example of a categorical variable is the airport column. In the airport column, we know that there should only be airport codes and that each of these codes should only have a 4 letter identifier. We also want to make sure that there are no inconsistencies - there should be a single way to represent each potential value. So let's take a closer look at this column of data.

In [ ]:
timed_df['airport'].unique()

One of the things that we see is that there are a bunch of upper case values and a bunch of lower case values. While in some cases this might be okay because these values might indicate different things, in our case, they should actually refer to the same airports. We should correct this now so that there are no unexpected behaviors during the analysis. 

This can be done with the upper() method. You'll notice that I have to put a .str before my method. This is because upper() is a string method and to access the string method we need the str attribute. Other methods such as date methods require this as well. We will explore this more in depth in a subsequent notebook.

In [ ]:
timed_df['airport'] = timed_df['airport'].str.upper()

Finally, let's move on to actually check if off and on events actually match up to the destination and departure airports. 

From the data, it seems that the airport column, which denotes where an event is being recorded, should be the same as the departure or destination airport columns depending on whether it is a take off or landing, respectively.

We see below that there are a number of data points where this is not the case. We cannot identify the source of this error fromt the data alone - it may be an bad sensor or a mistake in reporting. In either case, let's look at some examples and decide what to do with them. 

In [ ]:
index = (timed_df['destination_airport'] == timed_df['airport']) & (timed_df['destination_airport'] != timed_df['departure_airport']) & (timed_df['event'] == 'off')
timed_df[index][['airport','call_sign','time','event','departure_airport','destination_airport']].head()

In order to only look at a portion of the columns, we subsetted the data with a double brackets. This subsets only the columns that are listed. Secondly, you'll notice that conditional statement that we assign to the index is a bit more complicated this time. There are three parts to it: (1) does the destination airport match the airport column; (2) do the departure and destination airports not match; and (3) is the event a takeoff.

So we found all the takeoffs that seemed to have the wrong departure airport. Can we do this for all the landings with an inconsistent destination airport as well?

In [ ]:
# Answer
index = (timed_df['departure_airport'] == timed_df['airport']) & (timed_df['event'] == 'on') & (timed_df['destination_airport'] != timed_df['departure_airport'])
timed_df[index][['airport','call_sign','time','event','departure_airport','destination_airport']].head()

Based on inspection, it is possible that most of these records have their departure and destination airports reversed. However, we have no way to verify this with our current data, and we see that there only a small number of cases compared to the total amount of cases, so let's drop them. Why don't you give it a try? Hint: You might need the bitwise operator for or (|) when making your conditional statement to subset the data. 

In [ ]:
# Answer
index = ((timed_df['destination_airport'] == timed_df['airport']) & (timed_df['event'] == 'on')) | ((timed_df['departure_airport'] == timed_df['airport']) & (timed_df['event'] == 'off'))
clean_df = timed_df[index].copy()

Great! Now that we have finished cleaning the data, we can finally get around to exploring it and seeing what interesting things we can learn!